# KRX 데이터 조회 - 전일급등주
- description: 가즈아
- layout: post
- categories: [경제]
- toc: true

In [1]:
#hide
!pip install pandas pandasql xlrd

  Created wheel for pandasql: filename=pandasql-0.7.3-cp36-none-any.whl size=26819 sha256=7bacb1cb5b7451791ca2679364678b5de8cb42499c91e2f99a1f5948ecb00d88
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [2]:
# hide
import requests
import pandas as pd
from pandas.tseries.offsets import BDay
from io import StringIO
from io import BytesIO
import json
import datetime

왜 이렇게 만들었는지 모르겠는데 KRX는 한번 꼬아서 다운로드 받는다. 이러면 내가 못할 줄 알고?

In [3]:
def downloadKrxData(cd):
    resp = requests.post(
        'http://file.krx.co.kr/download.jspx', 
        headers={
            'User-Agent': 'Mozilla/5.0', 
            'Referer': 'http://marketdata.krx.co.kr/',
        }, 
        params={'code': cd},
    )
    tp = resp.headers['Content-Disposition'].split('.')[1]
    if tp == 'csv':
        return pd.read_csv(StringIO(resp.text))
    if tp == 'xls':
        return pd.read_excel(BytesIO(resp.content))

## KRX 휴장일을 반영한 전영업일 
- KRX 01023 휴장일 http://marketdata.krx.co.kr/mdi#document=01100305

In [4]:
def closingDays(year):
    resp = requests.get(
        'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx', 
        headers={'User-Agent': 'Mozilla/5.0'}, 
        params={
            'name': 'fileDown',
            'filetype': 'xls',
            'url': 'MKD/01/0110/01100305/mkd01100305_01',
            'search_bas_yy': year,
            'gridTp': 'KRX',
            'pagePath': '/contents/MKD/01/0110/01100305/MKD01100305.jsp',
        },
    )
    return downloadKrxData(resp.text)['일자 및 요일']

def getPreviousBizDay(today):
    pday = datetime.datetime.strptime(today, '%Y%m%d')
    yyyy = pday.year
    cds = closingDays(yyyy)

    while True:
        pday = pday - BDay(1)
        if yyyy != pday.year:
            yyyy = pday.year
            cds = pd.concat([cds, closingDays(yyyy)])
        if pday.strftime("%Y-%m-%d") not in cds.values:
            return pday.strftime("%Y%m%d")

## 전일 등락
- KRX 80037 전체종목 등락률 http://marketdata.krx.co.kr/mdi#document=13020102

In [5]:
def soaredStocks(dt):
    print(dt)
    resp = requests.get(
        'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx', 
        headers={'User-Agent': 'Mozilla/5.0'}, 
        params={
            'name': 'fileDown',
            'filetype': 'csv',
            'url': 'MKD/13/1302/13020102/mkd13020102',
            'ind_tp': 'ALL',
            'period_strt_dd': dt,
            'period_end_dd': dt,
            'pagePath': '/contents/MKD/13/1302/13020102/MKD13020102.jsp',
        },
    )
    return downloadKrxData(resp.text)

In [6]:
# hide
pd.set_option('display.max.colwidth', 150)

## 전 영업일 15% 이상 오른 종목

In [7]:
dt = getPreviousBizDay(datetime.datetime.now().strftime('%Y%m%d'))
df_soarings = soaredStocks(dt)

20201026


In [8]:
# hide
df_15 = df_soarings[df_soarings.등락률 >= 15].sort_values('등락률', ascending=False)
df_15['네이버주식'] = 'https://finance.naver.com/item/main.nhn?code=' + df_15['종목코드']
df_15

,종목코드,종목명,시작일기준가(원),종료일종가(원),대비,등락률,거래량(주),거래대금(원),네이버주식
422,008775,호텔신라우,"64,400","83,700","19,300",29.97,"82,225","6,882,213,200",https://finance.naver.com/item/main.nhn?code=008775
1055,050090,휘닉스소재,594,772,178,29.97,"2,041,795","1,576,265,740",https://finance.naver.com/item/main.nhn?code=050090
419,008700,아남전자,"1,905","2,475",570,29.92,"111,007,422","262,699,739,330",https://finance.naver.com/item/main.nhn?code=008700
747,02826K,삼성물산우B,"95,100","123,500","28,400",29.86,"579,727","71,394,674,500",https://finance.naver.com/item/main.nhn?code=02826K
1490,090710,휴림로봇,978,"1,270",292,29.86,"33,652,590","40,161,329,140",https://finance.naver.com/item/main.nhn?code=090710
2345,304840,피플바이오,"26,650","34,600","7,950",29.83,"7,114,876","221,196,968,650",https://finance.naver.com/item/main.nhn?code=304840
900,038010,제일테크노스,"4,430","5,750","1,320",29.80,"4,813,626","26,010,386,575",https://finance.naver.com/item/main.nhn?code=038010
2079,224060,코디엠,245,318,73,29.80,"156,012,012","47,783,465,961",https://finance.naver.com/item/main.nhn?code=224060
2405,323990,박셀바이오,"24,600","31,700","7,100",28.86,"10,137,430","294,918,851,100",https://finance.naver.com/item/main.nhn?code=323990
649,019770,서연탑메탈,"4,585","5,850","1,265",27.59,"17,596,630","98,033,346,915",https://finance.naver.com/item/main.nhn?code=019770


In [9]:
import pandasql as ps
ps.sqldf('''
    select a.*,
           'https://finance.naver.com/item/main.nhn?code=' || a.종목코드 as '네이버주식'
      from df_soarings a
     where 등락률 >= 15
  order by 등락률 desc
''')

,종목코드,종목명,시작일기준가(원),종료일종가(원),대비,등락률,거래량(주),거래대금(원),네이버주식
0,008775,호텔신라우,"64,400","83,700","19,300",29.97,"82,225","6,882,213,200",https://finance.naver.com/item/main.nhn?code=008775
1,050090,휘닉스소재,594,772,178,29.97,"2,041,795","1,576,265,740",https://finance.naver.com/item/main.nhn?code=050090
2,008700,아남전자,"1,905","2,475",570,29.92,"111,007,422","262,699,739,330",https://finance.naver.com/item/main.nhn?code=008700
3,02826K,삼성물산우B,"95,100","123,500","28,400",29.86,"579,727","71,394,674,500",https://finance.naver.com/item/main.nhn?code=02826K
4,090710,휴림로봇,978,"1,270",292,29.86,"33,652,590","40,161,329,140",https://finance.naver.com/item/main.nhn?code=090710
5,304840,피플바이오,"26,650","34,600","7,950",29.83,"7,114,876","221,196,968,650",https://finance.naver.com/item/main.nhn?code=304840
6,038010,제일테크노스,"4,430","5,750","1,320",29.80,"4,813,626","26,010,386,575",https://finance.naver.com/item/main.nhn?code=038010
7,224060,코디엠,245,318,73,29.80,"156,012,012","47,783,465,961",https://finance.naver.com/item/main.nhn?code=224060
8,323990,박셀바이오,"24,600","31,700","7,100",28.86,"10,137,430","294,918,851,100",https://finance.naver.com/item/main.nhn?code=323990
9,019770,서연탑메탈,"4,585","5,850","1,265",27.59,"17,596,630","98,033,346,915",https://finance.naver.com/item/main.nhn?code=019770
